In [14]:
# Install ElasticSearch
!pip install Elasticsearch
!service elasticsearch start

 * Starting Elasticsearch Server
 * Already running.
   ...done.


In [15]:
# import library

from elasticsearch import Elasticsearch
import json
import tqdm
import pandas as pd

In [99]:
class Elastic_retriever():

    def __init__(self, wiki_json_path, hundred = False, index_name = None):
        self.wiki_list = [] # saves text and document id only
        
        # load wikipedia data and drop needless informations 
        with open(wiki_json_path, "r", encoding = "utf-8") as f:
            wiki = json.load(f)

            if hundred:
                len_wiki = 100
            else:
                len_wiki = len(wiki)

            for ind in range(len_wiki):
                temp_wiki = wiki[str(ind)]
                self.wiki_list.append({"text":"!!" + temp_wiki["title"] + "!!" + temp_wiki["text"], "document_id" : temp_wiki["document_id"], "title" : temp_wiki["title"]})

        del wiki # for memory usage
        self.es = Elasticsearch("localhost:9200")

        if index_name is not None:
            self.index_name = index_name
        else:
            self.index_name = 'klue_mrc_wikipedia_index'

    def _create_indice(self, index_config = None):

        if index_config is None:
            index_config = {
                "settings": {
                    "analysis": {
                        "analyzer": {
                            "standard_analyzer": {
                                "type": "standard"
                            }
                        }
                    }
                },
                "mappings": {
                    "dynamic": "strict", 
                    "properties": {
                        "title" : {"type" :"text"},
                        "document_id": {"type": "long"},
                        "text": {"type": "text", "analyzer": "standard_analyzer"}
                        }
                    }
                }

        if self.es.indices.exists(index=self.index_name):
            self.es.indices.delete(index=self.index_name)

        self.es.indices.create(index=self.index_name, body=index_config, ignore=400)

    def _populate_index(self):
        
        for i in tqdm.tqdm(range(len(self.wiki_list))):
            self.es.index(index = self.index_name, id = i, body = self.wiki_list[i])
        
    def config_and_index(self, index_name = None, index_config = None):
        self._create_indice(index_name, index_config)
        self._populate_index(index_name)

    def search(self, query, num_return, index_name = None):
        if index_name is None:
            index_name = 'klue_mrc_wikipedia_index'
        answer = self.es.search(index=index_name, q = query, size = num_return)
        return answer

In [100]:
index_config = {
        "settings": {
            "analysis": {
                "filter":{
                    "my_stop_filter": { 
                        "type" : "stop",
                        "stopwords_path" : "stop_words.txt" # /etc/elastic안에 txt파일이 존재해야 댑니다
                    }
                },
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer", # 노리 형태소 깔아야대는데 에러나면 맨위에 참고해서 깔기
                        "decompound_mode": "none",
                        "filter" : ["my_stop_filter"]# 위에서 정의한 stopword
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "strict", # 먼지 잘모르겟
            "properties": {
                "title" : {"type" :"text"},
                "document_id": {"type": "long",},
                "text": {"type": "text", "analyzer": "nori_analyzer"}
                }
            }
        }

In [101]:
wiki_json_path = "/opt/ml/code/preprocessed_json_v3.json"

retriever = Elastic_retriever(wiki_json_path)
retriever._create_indice(index_config = index_config)
retriever._populate_index()

<ipython-input-99-599e7807718c>:53: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  self.es.indices.create(index=self.index_name, body=index_config, ignore=400)
  0%|          | 0/60613 [00:00<?, ?it/s]<ipython-input-99-599e7807718c>:58: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  self.es.index(index = self.index_name, id = i, body = self.wiki_list[i])
100%|██████████| 60613/60613 [04:43<00:00, 214.08it/s]


In [88]:
def convert(train_path, valid_path):
    # make a list of dictionary
    total_data = [] # {context, question, document_id}

    train_df = pd.read_csv(train_path, index_col = 0)
    valid_df = pd.read_csv(valid_path, index_col = 0)

    total_df = pd.concat([train_df, valid_df])

    for i in range(len(total_df)):
        temp_data = total_df.iloc[i]
        total_data.append({"text":temp_data.context, "question":temp_data.question, "document_id": temp_data.document_id})

    return total_data

train_data = "/opt/ml/code/train_dataset_no_tilde.csv"
valid_data = "/opt/ml/code/valid_dataset_no_tilde.csv"

total_data = convert(train_data, valid_data)

In [102]:
import tqdm
import re
from konlpy.tag import Kkma

tokenizer = Kkma()

def show_the_result_with_tokenizer(retriever, total_data, tokenizer):

    total_data_len = len(total_data)
    results = [0]*21
    token_results = [0]*21

    for ind in tqdm.tqdm(range(total_data_len)):

        temp_data = total_data[ind]
        query = re.sub("~","-", temp_data["question"])
        query = re.sub("/","", query)
        # extract nouns only

        pos = tokenizer.pos(query)
        pos = [x[0] for x in pos if "NN" in x[1] or "UN" in x[1]]

        document_id = temp_data["document_id"]
        hit_ones = retriever.search(query, 20)["hits"]["hits"]

        if hit_ones: #만약 검출이 되었다면
            hit_ids = [hit_one["_source"]["document_id"] for hit_one in hit_ones]
            hit_contexts = [hit_one["_source"]["text"] for hit_one in hit_ones]

            # 가장 일치가 많이 된 text 검색
            best_match = (20,0) # index and n_match
            for i, context in enumerate(hit_contexts):
                temp_match = 0
                for p in pos:
                    if p in context:
                        temp_match += 1
                if temp_match > best_match[1]:
                    best_match = (i, temp_match)

            if document_id in hit_ids:
                found_index = hit_ids.index(document_id)
                results[found_index] +=1 
            else:
                results[-1] += 1

        token_results[best_match[0]] +=1

    return results, token_results

def show_the_result(retriever, total_data):

    results = [0]*21

    total_data_len = len(total_data)
    
    for ind in tqdm.tqdm(range(total_data_len)):

        temp_data = total_data[ind]
        query = re.sub("~","-", temp_data["question"])
        query = re.sub("/","", query)

        document_id = temp_data["document_id"]

        hit_ones = retriever.search(query, 20)["hits"]["hits"]

        if hit_ones: #만약 검출이 되었다면
            result = [hit_one["_source"]["document_id"] for hit_one in hit_ones]
            
            if document_id in result:
                found_index = result.index(document_id)
                results[found_index] +=1 
            else:
                results[-1] += 1

    return results

In [103]:
#results = show_the_result(retriever, total_data)
results, token_results = show_the_result_with_tokenizer(retriever, total_data, tokenizer)

 31%|███       | 1283/4192 [00:43<01:18, 36.98it/s]

In [97]:
print(results)
print(token_results)

[2329, 860, 179, 146, 56, 57, 29, 30, 24, 24, 19, 11, 22, 17, 16, 12, 20, 7, 12, 7, 314]
[2957, 326, 198, 126, 91, 73, 41, 40, 54, 44, 36, 28, 26, 34, 24, 21, 20, 15, 19, 18, 1]


In [98]:
a = retriever.search("최재수는 2012년에 어디로 이적했는가?", 20)
b = retriever.search("괴수로부터 메구밍을 구해준 사람은 누구인가?", 20)

import pprint

pprint.pprint(a)
pprint.pprint(b)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '24045',
                    '_index': 'klue_mrc_wikipedia_index',
                    '_score': 19.837315,
                    '_source': {'document_id': 24045,
                                'text': '!!최재수!!광주에서 제대 후 2010시즌을 앞두고 서울로 '
                                        '복귀했다. 하지만 2009년 겨울 최효진, 현영민 등 국가 대표급 '
                                        '사이드 풀백들이 대거 영입되면서 그의 입지는 또 다시 미미한 상황이 '
                                        '되었다. 이에 당시 사이드 자원의 공백으로 곤란을 겪던 울산으로 '
                                        '2년 계약을 맺으며 이적했다. 그러나 주전으로 입지를 다지기도 전에 '
                                        '서울 시절 그의 발목을 잡았던 김동진 역시 울산으로 오게 되면서 '
                                        '다시 위기에 빠졌다. 하지만 김동진이 부상으로 주춤하는 사이 자신의 '
                                        '능력을 마음껏 발휘하였고, 마침내 김동진을 밀어내고 울산의 주전을 '
                                        '꿰차는 데 성공했다. 2010년 29경기에 출전해서 6도움을 올리며 '
                       

In [78]:
print(sum(results[:5]))
print(sum(token_results[5:]))

3534
494


In [10]:
def pretty_result(result):
    total = sum(result)
    top_1 = result[0]
    top_5 = sum(result[:2])
    top_10 = sum(result[:3])
    top_20 = sum(result[:4])

    print(f"===Retrieval Result===\n")
    print(f"top 1 : {top_1*100/total}%")
    print(f"top 5 : {top_5*100/total}%")
    print(f"top 10 : {top_10*100/total}%")
    print(f"top 20 : {top_20*100/total}%")
    print(f"failed to predict : {result[-1]*100/total}%")

In [84]:
a = retriever.search("최재수는 2012년에 어디로 이적했는가?", 100)

import pprint

pprint.pprint(a)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '40533',
                    '_index': 'klue_mrc_wikipedia_index',
                    '_score': 6.4549646,
                    '_source': {'document_id': 40533,
                                'text': '이클립스는 아래와 같이 우수상 및 표창을 받았다 * Charles '
                                        'S. Roberts Best Science-Fiction or '
                                        'Fantasy Board Wargame Nominee (2011) '
                                        '* Jogo do Ano Nominee (2011) * Golden '
                                        'Geek Best Board Game '
                                        'Artwork/Presentation Nominee (2012) * '
                                        'Golden Geek Best Innovative Board '
                                        'Game Nominee (2012) * Golden Geek '
                                        'Best Strategy Board Game Nominee '
                                        '(2012)

In [12]:
pretty_result(results)

===Retrieval Result===

top 1 : 54.87950369840134%
top 5 : 75.1133381054641%
top 10 : 79.5275590551181%
top 20 : 83.0112145072775%
failed to predict : 8.136482939632545%


# 실험 

- 명사추출해서 명사가 있으면 가산점 주자

In [20]:
!pip install konlpy

from konlpy.tag import Kkma